In [1]:
numeric_stats = ['derived_fg', 'derived_ft', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
fantasy_stats = ['derived_fg', 'derived_ft','TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']
negative_stats = ['FTA', 'TOV', 'FGA']

In [2]:
import unidecode

def name_transform(name):
    name = name.split('\\')[0].strip()
    name = unidecode.unidecode(name)
    transforms = {}
    transforms['Marcus Morris'] = 'Marcus Morris Sr.'
    transforms['Otto Porter'] = 'Otto Porter Jr.'
    if name in transforms:
        return transforms[name]
    return name
name_transform('Otto Porter')

'Otto Porter Jr.'

In [3]:
# Import stats CSV
import pandas as pd

arr = pd.read_csv('current_season.csv')
arr['Player'] = arr['Player'].transform(name_transform)
arr = arr.drop(['Rk', 'GS', 'Age', 'Pos', 'FG%', 'FT%', '2P%', 'eFG%', '3PA', '3P%', '2P', '2PA', 'Tm'], axis=1)
arr = arr.groupby(arr['Player']).sum()
arr['FT%'] = arr['FT'] * 100 / arr['FTA']
arr['FG%'] = arr['FG'] * 100 / arr['FGA']
arr['derived_fg'] = arr['FG'] * (arr['FG%'] - arr['FG%'].mean())
arr['derived_ft'] = arr['FT'] * (arr['FT%'] - arr['FT%'].mean())
arr[['TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']] = arr[['TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']].div(arr['G'], axis=0)

In [6]:
print(arr[fantasy_stats].corr().to_json())

{"derived_fg":{"derived_fg":1.0,"derived_ft":0.035092301,"TRB":0.529424214,"AST":0.194769702,"STL":0.2058326544,"BLK":0.339499336,"TOV":0.328099994,"3P":0.061836175,"PTS":0.4699048064},"derived_ft":{"derived_fg":0.035092301,"derived_ft":1.0,"TRB":-0.0243886008,"AST":0.4048544335,"STL":0.2046268748,"BLK":-0.0749430107,"TOV":0.298600611,"3P":0.4654155927,"PTS":0.4794027672},"TRB":{"derived_fg":0.529424214,"derived_ft":-0.0243886008,"TRB":1.0,"AST":0.4148955457,"STL":0.3740432297,"BLK":0.5535691841,"TOV":0.5666131839,"3P":0.2192159988,"PTS":0.609973406},"AST":{"derived_fg":0.194769702,"derived_ft":0.4048544335,"TRB":0.4148955457,"AST":1.0,"STL":0.5488814612,"BLK":0.0670505665,"TOV":0.7513162495,"3P":0.4838632326,"PTS":0.7114900391},"STL":{"derived_fg":0.2058326544,"derived_ft":0.2046268748,"TRB":0.3740432297,"AST":0.5488814612,"STL":1.0,"BLK":0.1839274425,"TOV":0.4997066435,"3P":0.423988508,"PTS":0.5620030955},"BLK":{"derived_fg":0.339499336,"derived_ft":-0.0749430107,"TRB":0.5535691841,"